In [1]:
!pip install numpy

In [2]:
!pip install scikit-learn

In [3]:
!pip install tensorflow

In [4]:
!pip install Pillow

In [5]:
!pip install opencv-python

In [6]:
import tensorflow as tf

print("Is Built with CUDA: ", tf.test.is_built_with_cuda())

2023-11-26 16:41:00.700560: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-26 16:41:00.765059: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Is Built with CUDA:  True


In [7]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  4


In [8]:
print(tf.__version__)

2.13.0


In [9]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [10]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from PIL import Image
import os

def load_images_and_masks(image_dir, mask_dir, limit=10):
    image_paths = sorted([
        os.path.join(image_dir, fname)
        for fname in os.listdir(image_dir)
        if fname.endswith('.png') and fname.startswith("x")
    ])[:limit]

    mask_paths = sorted([
        os.path.join(mask_dir, fname)
        for fname in os.listdir(mask_dir)
        if fname.endswith('.png') and fname.startswith("y")
    ])[:limit]

    images = [np.array(Image.open(path).convert("L")) / 255.0 for path in image_paths]
    images = [image.reshape(512, 512, 1) for image in images]
    masks = [np.array(Image.open(path).convert("L")) for path in mask_paths]

    for i, mask in enumerate(masks):
        masks[i][mask == 127] = 1  # Class 1
        masks[i][mask == 255] = 2  # Class 2
        masks[i] = to_categorical(masks[i], num_classes=3)

    return np.array(images), np.array(masks)

X, Y = load_images_and_masks("x", "y", 1032)

In [11]:
X.shape

(1032, 512, 512, 1)

In [12]:
Y.shape

(1032, 512, 512, 3)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [14]:
print(X.shape)
print(f"Train size: {X_train.shape}, {y_train.shape}")
print(f"Val size: {X_val.shape}, {y_val.shape}")

(1032, 512, 512, 1)
Train size: (825, 512, 512, 1), (825, 512, 512, 3)
Val size: (103, 512, 512, 1), (103, 512, 512, 3)


In [15]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, concatenate, ConvLSTM2D, Reshape
from tensorflow.keras.models import Model

def r2_block(x, filters, kernel_size=3):
    # Recurrent convolutional block
    # Adding a time dimension (of size 1) for ConvLSTM2D
    x = Reshape((1, x.shape[1], x.shape[2], x.shape[3]))(x)
    conv = ConvLSTM2D(filters, kernel_size, padding='same', kernel_initializer='he_normal', return_sequences=False)(x)
    # Reshape back to the regular dimensions for Conv2D layers
    
    conv = Reshape((conv.shape[1], conv.shape[2], filters))(conv)
    conv = Conv2D(filters, kernel_size, activation='relu', padding='same', kernel_initializer='he_normal')(conv)
    return conv

def r2unet(input_size=(512, 512, 1), num_classes=3):
    inputs = Input(input_size)

    # Layer 1 with R2 block
    conv1 = r2_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Layer 2 with R2 block
    conv2 = r2_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    # Layer 3 with R2 block
    conv3 = r2_block(pool2, 256)

    drop3 = Dropout(0.5)(conv3)  # Adding dropout after the deepest layer

    # Up-sampling and Merge Layer 3
    up7 = UpSampling2D(size=(2, 2))(drop3)
    up7 = r2_block(up7, 128)
    merge7 = concatenate([conv2, up7], axis=3)
    conv7 = r2_block(merge7, 128)

    # Up-sampling and Merge Layer 2
    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = r2_block(up8, 64)
    merge8 = concatenate([conv1, up8], axis=3)
    conv8 = r2_block(merge8, 64)

    # Final Convolutional Layer
    conv9 = Conv2D(num_classes, 3, padding='same', kernel_initializer='he_normal')(conv8)
    conv10 = Conv2D(num_classes, 1, activation='softmax')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model


In [16]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Recall, Precision, MeanIoU
from tensorflow.keras.losses import categorical_crossentropy


smooth = 1e-6  # To avoid division by zero
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6  # To avoid division by zero
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=-1)
    sum_ = tf.keras.backend.sum(tf.keras.backend.abs(y_true) + tf.keras.backend.abs(y_pred), axis=-1)
    iou = (intersection + smooth) / (sum_ - intersection + smooth)
    return iou

def combined_dice_crossentropy_loss(y_true, y_pred, dice_loss_weight=0.5):
    # Dice Loss
    dice_loss_value = dice_loss(y_true, y_pred)

    # Categorical Cross-Entropy Loss
    cross_entropy_loss_value = categorical_crossentropy(y_true, y_pred)

    # Combined Loss
    combined_loss = dice_loss_weight * dice_loss_value + (1 - dice_loss_weight) * cross_entropy_loss_value
    return combined_loss



strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model = r2unet()
    optimizer = Adam(learning_rate=1e-4)  # Example learning rate
    model.compile(optimizer=optimizer, loss=combined_dice_crossentropy_loss, metrics=["accuracy", dice_coefficient, iou])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


2023-11-26 16:41:20.652174: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20609 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9
2023-11-26 16:41:20.654802: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 20609 MB memory:  -> device: 1, name: NVIDIA L4, pci bus id: 0000:00:04.0, compute capability: 8.9
2023-11-26 16:41:20.656666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 20609 MB memory:  -> device: 2, name: NVIDIA L4, pci bus id: 0000:00:05.0, compute capability: 8.9
2023-11-26 16:41:20.658987: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 20609 MB memory:  -> device: 3, name: NVIDIA L4, pci bus id: 0000:00:06.0, compute capability: 8.9


Number of devices: 4


In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import datetime


log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Fit the model
callbacks = [
    ModelCheckpoint('model_best.h5', verbose=1, save_best_only=True),
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    tensorboard_callback
]

results = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8, epochs=200,
    callbacks=callbacks
)

Epoch 1/200
INFO:tensorflow:Collective all_reduce tensors: 39 all_reduces, num_devices = 4, group_size = 4, implementation = CommunicationImplementation.NCCL, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/devi

2023-11-26 16:42:27.344487: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-11-26 16:42:45.573929: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-11-26 16:42:46.678199: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-11-26 16:42:47.664475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-11-26 16:42:48.477218: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-26 16:42:48.622635: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8900
2023-11-26 16:42:55.452088: I tensorflow/compiler/xla/service/service.cc:168] XLA se

104/104 [==============================] - ETA: 0s - loss: 0.5405 - accuracy: 0.7645 - dice_coefficient: 0.6128 - iou: 0.5175
Epoch 1: val_loss improved from inf to 0.39189, saving model to model_best.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


104/104 [==============================] - 216s 1s/step - loss: 0.5405 - accuracy: 0.7645 - dice_coefficient: 0.6128 - iou: 0.5175 - val_loss: 0.3919 - val_accuracy: 0.7753 - val_dice_coefficient: 0.7138 - val_iou: 0.6463 - lr: 1.0000e-04
Epoch 2/200
104/104 [==============================] - ETA: 0s - loss: 0.3650 - accuracy: 0.7786 - dice_coefficient: 0.7381 - iou: 0.6744
Epoch 2: val_loss improved from 0.39189 to 0.36057, saving model to model_best.h5
104/104 [==============================] - 116s 1s/step - loss: 0.3650 - accuracy: 0.7786 - dice_coefficient: 0.7381 - iou: 0.6744 - val_loss: 0.3606 - val_accuracy: 0.7910 - val_dice_coefficient: 0.7249 - val_iou: 0.6566 - lr: 1.0000e-04
Epoch 3/200
104/104 [==============================] - ETA: 0s - loss: 0.3367 - accuracy: 0.7855 - dice_coefficient: 0.7593 - iou: 0.7008
Epoch 3: val_loss improved from 0.36057 to 0.33070, saving model to model_best.h5
104/104 [==============================] - 115s 1s/step - loss: 0.3367 - accuracy:

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy, d, i = model.evaluate(X_test, y_test, verbose=1)

# Print out the performance metrics
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Dice Co-efficient: {d}")
print(f"Test iou: {i}")

2/4 [==============>...............] - ETA: 3s - loss: 0.0990 - accuracy: 0.9511 - dice_coefficient: 0.9377 - iou: 0.9197 

# **Split**

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Recall, Precision, MeanIoU
from tensorflow.keras.losses import categorical_crossentropy


smooth = 1e-6  # To avoid division by zero
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6  # To avoid division by zero
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def iou(y_true, y_pred):
    intersection = tf.keras.backend.sum(tf.keras.backend.abs(y_true * y_pred), axis=-1)
    sum_ = tf.keras.backend.sum(tf.keras.backend.abs(y_true) + tf.keras.backend.abs(y_pred), axis=-1)
    iou = (intersection + smooth) / (sum_ - intersection + smooth)
    return iou

def combined_dice_crossentropy_loss(y_true, y_pred, dice_loss_weight=0.5):
    # Dice Loss
    dice_loss_value = dice_loss(y_true, y_pred)

    # Categorical Cross-Entropy Loss
    cross_entropy_loss_value = categorical_crossentropy(y_true, y_pred)

    # Combined Loss
    combined_loss = dice_loss_weight * dice_loss_value + (1 - dice_loss_weight) * cross_entropy_loss_value
    return combined_loss

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import cv2


def get_input_array(image_path):
  array = np.array(Image.open(image_path).convert("L").resize((512, 512), Image.LANCZOS))
  array = array.reshape(512, 512, 1)
  array = array / 255
  array = np.expand_dims(array, axis = 0) # Adding batch.
  return array

def get_probability_distro(input_array):
  model_path = 'model_best.h5'
  x = {
      "combined_dice_crossentropy_loss": combined_dice_crossentropy_loss,
      "iou": iou,
      "dice_coefficient": dice_coefficient
  }
  model = load_model(model_path, custom_objects=x)
  predictions = model.predict(input_array)

  probability_distro = predictions[0]
  return probability_distro

def get_segmentation(probability_distro):
  return np.argmax(probability_distro, axis = -1)

def get_rgb_image(segmentation):
  rgb_image = np.zeros((*segmentation.shape, 3), dtype=np.uint8)

  colors = {
      0: [0, 0, 0], # Black - BG
      1: [127, 127, 127], # Grey - VAT
      2: [255, 255, 255] # White - SAT
  }

  for class_index, color in colors.items():
    rgb_image[segmentation == class_index] = color

  return rgb_image

def get_grayscale_image(segmentation):
    grayscale_image = np.zeros(segmentation.shape, dtype=np.uint8)

    intensities = {
        0: 0,   # Black - BACKGROUND
        1: 127, # Grey - VISCERAL_FAT
        2: 255  # White - SUPERFICIAL_FAT
    }

    for class_index, intensity in intensities.items():
        grayscale_image[segmentation == class_index] = intensity

    return grayscale_image

import matplotlib.pyplot as plt

def clean_gray_scale_image(gray_scale_image):
    """
    This function converts only the white pixels within the interior of the ring to grey,
    while preserving the white border and existing grey areas.
    """
    # Define class values
    BACKGROUND = 0  # Black
    VISCERAL_FAT = 127  # Grey
    SUPERFICIAL_FAT = 255  # White

    # Step 1: Create a mask for the white (superficial fat) areas
    white_mask = gray_scale_image == SUPERFICIAL_FAT

    # Initialize a mask that will be used to fill contours
    mask = np.zeros_like(gray_scale_image, dtype=np.uint8)

    # Step 2: Find all contours on the white mask
    contours, hierarchy = cv2.findContours(white_mask.astype(np.uint8), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # Step 3: Fill the white areas that are not border (i.e., have a parent contour which is the border)
    for i, contour in enumerate(contours):
        # Check if the contour has a parent contour
        if hierarchy[0, i, 3] != -1:  # Has a parent contour
            cv2.drawContours(mask, [contour], -1, 1, -1)

    # Update segmentation image: convert the white to grey where mask is 1 and original pixel is not BACKGROUND
    gray_scale_image = np.where((mask == 1) & (gray_scale_image != BACKGROUND), VISCERAL_FAT, gray_scale_image)

    return gray_scale_image

In [ ]:
input_array = get_input_array("x/x_1.png")
probability_distro = get_probability_distro(input_array)
segmentation = get_segmentation(probability_distro)

In [ ]:
gray_scale_image = get_grayscale_image(segmentation)
final_image = clean_gray_scale_image(gray_scale_image.copy())

In [ ]:
# Display the original and converted images
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(gray_scale_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Final image')
plt.imshow(final_image, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# def convert_interior_white_to_grey(segmentation):
#     """
#     This function converts only the white pixels within the interior of the ring to grey,
#     while preserving the white border and existing grey areas.
#     """
#     # Define class values
#     BACKGROUND = 0  # Black
#     VISCERAL_FAT = 128  # Grey
#     SUPERFICIAL_FAT = 255  # White

#     # Step 1: Create a mask for the white (superficial fat) areas
#     white_mask = segmentation == 0

#     # Step 2: Find all contours on the white mask
#     contours, hierarchy = cv2.findContours(white_mask.astype(np.uint8), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

#     # Step 3: Fill the white areas that are not border (i.e., have a parent contour which is the border)
#     for i, contour in enumerate(contours):
#         # Check if the contour has a parent contour
#         if hierarchy[0, i, 3] != -1:  # Has a parent contour
#             cv2.drawContours(segmentation, [contour], -1, VISCERAL_FAT, -1)

#     return segmentation

# image = np.array(Image.open("a.png").convert("L"))
# # Apply the conversion function to the original segmentation
# converted_interior_segmentation = convert_interior_white_to_grey(image.copy())

# # Display the original and converted images
# plt.figure(figsize=(10, 5))

# plt.subplot(1, 2, 1)
# plt.title('Original Image')
# plt.imshow(image, cmap='gray')
# plt.axis('off')

# plt.subplot(1, 2, 2)
# plt.title('Converted Interior Segmentation')
# plt.imshow(converted_interior_segmentation, cmap='gray')
# plt.axis('off')

# plt.tight_layout()
# plt.show()
